In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold,cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC

In [33]:
openCloseData=pd.read_csv("openCloseData8.csv")
openCloseData.head()

,Unnamed: 0,PostId,UserId,Comment,Body,Title,Tags,Reputation,experienceInTime,WebsiteUrl,...,locationFilled,aboutMeFilled,profileImageUrlFilled,tagWeightSum,gunningFog,fleschReadingEase,daleChallReadabilityScore,smogIndex,colemanLiauIndex,userProfileFilled
0,0,21170255,5921,102,"<p>I tried to Google for this, but was (surpri...",Can anyone point me to (or post) an array of e...,<javascript><html><arrays><tags>,17527,1953,NaN,...,1,1,0,-207.161031,71.70,-80.10,13.96,0.0,9.83,2
1,1,38290647,1601251,102,<p>I am at a decision fork where i have to pic...,RESTful API with either Tomcat or Node.js?,<java><node.js><rest><tomcat><typescript>,151,1425,NaN,...,0,0,0,-536.990588,52.99,-26.31,12.32,0.0,11.98,0
2,2,6496114,52924,102,<p>Does anyone know of a good tool to test Sch...,Tool for testing Schema.org markup,<testing><markup><schema.org>,1639,900,http://www.agendarecife.com,...,1,1,0,-160.004636,17.06,64.54,7.83,0.0,14.97,3
3,3,25982771,1968462,102,"<p>The <a href=""http://www.postgresql.org/docs...",Inserting into simple Postgres views,<sql><postgresql>,2669,620,https://katrinahoffert.github.io/,...,1,1,1,-188.646160,26.89,32.23,9.16,0.0,17.02,4
4,4,10142243,383479,102,<p>Which frameworks would you recommend for wr...,Frameworks for Unittests for Multithreading in...,<java><multithreading><unit-testing><concurrency>,245,648,NaN,...,0,0,0,-274.409048,21.25,45.43,8.34,0.0,9.41,0


In [34]:
print(openCloseData.shape)
openCloseData.drop(['Unnamed: 0'],inplace=True,axis=1)
print(openCloseData.shape)

(159996, 42)
(159996, 41)


In [35]:
openCloseData.columns

Index(['PostId', 'UserId', 'Comment', 'Body', 'Title', 'Tags', 'Reputation',
       'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views',
       'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount', 'closed',
       'lowerUpperRatio', 'sentenceCount', 'firstLineLength', 'codeLength',
       'urlCount', 'SOUrlCount', 'titleLengthInChars',
       'countOfInterrogativeSent', 'sentencesStartWithYouCount',
       'sentencesStartWithICount', 'shortWordCount', 'bodyWordCount',
       'punctuationCount', 'websiteUrlFilled', 'locationFilled',
       'aboutMeFilled', 'profileImageUrlFilled', 'tagWeightSum', 'gunningFog',
       'fleschReadingEase', 'daleChallReadabilityScore', 'smogIndex',
       'colemanLiauIndex', 'userProfileFilled'],
      dtype='object')

In [36]:
# closeData = openCloseData[80001:]
closeData = openCloseData[openCloseData['closed'] == 1]
closeData.shape

(79996, 41)

In [37]:
X = closeData.loc[:,['Comment', 'Reputation',
       'experienceInTime', 'Views',
       'Upvotes', 'Downvotes', 'tagCount',
       'lowerUpperRatio', 'sentenceCount', 'firstLineLength', 'codeLength',
       'urlCount', 'SOUrlCount', 'titleLengthInChars',
       'countOfInterrogativeSent', 'sentencesStartWithYouCount',
       'sentencesStartWithICount', 'shortWordCount', 'bodyWordCount',
       'punctuationCount', 'gunningFog',
       'fleschReadingEase', 'daleChallReadabilityScore', 'smogIndex',
       'colemanLiauIndex', 'userProfileFilled'] ]
X.replace(np.nan,0,inplace=True)  #replacing NaN to 0
X.lowerUpperRatio.replace(np.inf,0,inplace=True)  #replacing infinity to 0
print(X.shape)
print(type(X))

(79996, 26)
<class 'pandas.core.frame.DataFrame'>


In [38]:
X.head()

,Comment,Reputation,experienceInTime,Views,Upvotes,Downvotes,tagCount,lowerUpperRatio,sentenceCount,firstLineLength,...,sentencesStartWithICount,shortWordCount,bodyWordCount,punctuationCount,gunningFog,fleschReadingEase,daleChallReadabilityScore,smogIndex,colemanLiauIndex,userProfileFilled
0,102,17527,1953,2765,2005,119,4,26.153846,6,69,...,4,84,168,50,71.70,-80.10,13.96,0.0,9.83,2
1,102,151,1425,32,16,0,5,16.448276,10,75,...,1,51,113,19,52.99,-26.31,12.32,0.0,11.98,0
2,102,1639,900,208,75,1,3,18.916667,4,79,...,0,15,47,17,17.06,64.54,7.83,0.0,14.97,3
3,102,2669,620,658,282,11,2,12.272727,5,210,...,2,46,94,45,26.89,32.23,9.16,0.0,17.02,4
4,102,245,648,78,14,0,4,98.000000,2,92,...,0,31,68,11,21.25,45.43,8.34,0.0,9.41,0


In [39]:
y = X['Comment'] - 102
X.drop(['Comment'],inplace=True,axis=1)
print(X.shape, y.shape)
print(X.columns)

(79996, 25) (79996,)
Index(['Reputation', 'experienceInTime', 'Views', 'Upvotes', 'Downvotes',
       'tagCount', 'lowerUpperRatio', 'sentenceCount', 'firstLineLength',
       'codeLength', 'urlCount', 'SOUrlCount', 'titleLengthInChars',
       'countOfInterrogativeSent', 'sentencesStartWithYouCount',
       'sentencesStartWithICount', 'shortWordCount', 'bodyWordCount',
       'punctuationCount', 'gunningFog', 'fleschReadingEase',
       'daleChallReadabilityScore', 'smogIndex', 'colemanLiauIndex',
       'userProfileFilled'],
      dtype='object')


In [40]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X=scaler.fit_transform(X)
print(type(X))
X = pd.DataFrame(X, columns=['Reputation', 'experienceInTime', 'Views',
       'Upvotes', 'Downvotes', 'tagCount', 'lowerUpperRatio', 'sentenceCount', 'firstLineLength', 'codeLength',
       'urlCount', 'SOUrlCount', 'titleLengthInChars', 'countOfInterrogativeSent', 'sentencesStartWithYouCount',
       'sentencesStartWithICount', 'shortWordCount', 'bodyWordCount', 'punctuationCount', 'gunningFog',
       'fleschReadingEase', 'daleChallReadabilityScore', 'smogIndex', 'colemanLiauIndex', 'userProfileFilled'])

<class 'numpy.ndarray'>


In [41]:
#Train/Test Split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(55997, 25)
(55997,)
(23999, 25)
(23999,)


## AdaBoost Using Decision Tree(Default)

In [42]:
ada_boost_dec_tree = AdaBoostClassifier()
ada_boost_dec_tree.fit(X_train, y_train)
y_predicted = ada_boost_dec_tree.predict(X_test)
print('Accuracy: ' + str(np.mean(y_test==y_predicted))) 

Accuracy: 0.427101129214


In [43]:
print(classification_report(y_test,y_predicted))

             precision    recall  f1-score   support

          0       0.35      0.20      0.26      5925
          1       0.43      0.51      0.47      6074
          2       0.39      0.38      0.38      6011
          3       0.49      0.61      0.54      5989

avg / total       0.41      0.43      0.41     23999



## AdaBoost Using Linear SVM Classifier

In [44]:
ada_boost_svm = AdaBoostClassifier(LinearSVC(), algorithm='SAMME')
ada_boost_svm.fit(X_train, y_train)
y_predicted = ada_boost_svm.predict(X_test)
print('Accuracy: ' + str(np.mean(y_test==y_predicted)))

Accuracy: 0.40614192258


In [45]:
print(classification_report(y_test,y_predicted))

             precision    recall  f1-score   support

          0       0.37      0.19      0.25      5925
          1       0.39      0.56      0.46      6074
          2       0.36      0.22      0.28      6011
          3       0.46      0.64      0.54      5989

avg / total       0.39      0.41      0.38     23999



In [46]:
k_fold = KFold(n_splits=5)

In [47]:
# cross_val_score(ada_boost_svm,X,y,cv=k_fold,scoring='accuracy',n_jobs=-1)

## AdaBoost + Random Forest

In [48]:
from sklearn.ensemble import RandomForestClassifier

In [49]:
ada_boost_rand_forest = AdaBoostClassifier(RandomForestClassifier())
ada_boost_rand_forest.fit(X_train, y_train)
y_predicted = ada_boost_rand_forest.predict(X_test)
print('Accuracy: ' + str(np.mean(y_test==y_predicted)))

Accuracy: 0.406600275011


In [50]:
print(classification_report(y_test,y_predicted))

             precision    recall  f1-score   support

          0       0.34      0.26      0.29      5925
          1       0.41      0.44      0.42      6074
          2       0.36      0.33      0.34      6011
          3       0.48      0.60      0.53      5989

avg / total       0.40      0.41      0.40     23999



## AdaBoost Using Logistic Regression

In [51]:
from sklearn.linear_model import LogisticRegression

In [52]:
ada_boost_log_reg = AdaBoostClassifier(LogisticRegression())
ada_boost_log_reg.fit(X_train, y_train)
y_predicted = ada_boost_log_reg.predict(X_test)
print('Accuracy: ' + str(np.mean(y_test==y_predicted)))

Accuracy: 0.391099629151


In [53]:
print(classification_report(y_test,y_predicted))

             precision    recall  f1-score   support

          0       0.34      0.22      0.26      5925
          1       0.38      0.52      0.44      6074
          2       0.33      0.29      0.31      6011
          3       0.49      0.54      0.51      5989

avg / total       0.38      0.39      0.38     23999

